# PyWren RISECamp, 2017

## Data Analytics with PyWren

In this section, we will use PyWren explore a dataset of Wikipedia records.

## 0. The Data

We've prepared an S3 bucket with 20GB of Wikipedia traffic statistics data obtained from http://aws.amazon.com/datasets/4182. To make the analysis more feasible for the short time you're here, we've shortened the dataset to three days worth of data (May 5 to May 7, 2009; roughly 20G and 329 million entries). 

Let's take a look into the bucket with our dataset. We'll print a few files from a few files from our bucket.


In [ ]:
from training import list_keys_with_prefix, read_from_s3, wikipedia_bucket

filenames = list_keys_with_prefix(wikipedia_bucket, "wikistats_20090505_restricted-01/")
for filename in filenames[:20]:
    print(filename)

There are 74 files (2 of which are intentionally left empty). Each file consists of a list of records. Let's go take a look into the first file. This may take a while.

In [ ]:
# Read a file into memory, and split on newlines.
records = read_from_s3(wikipedia_bucket, wikistats_20090505_restricted-01/part-00001).split("\n")

print("The total number of records in this file is {}, but here are the first 20".format(len(records)))
for i in range(20):
    print records[i]

Each record has stats for a single Wikipedia page. The schema is:

`<date_time> <project_code> <page_name> <page_views> <page_size>`

- `<date_time>` specifies a date in YYYYMMDD-HHmmSS format (year, month, day, hour minute, second).
- `<project_code>` specifies the language the page is written in. For example, project code “en” indicates an English page. 
- `<page_title>` gives the page title.
- `<page_views>` gives the number of page views in the hour-long time slot starting at `<data_time>`. 
- `<page_size>` gives the size in bytes of the  page.

Now that we have a better understanding of the structure of our data, we can start running some interesting queries. 

Because the data are so large, it takes us quite a while to load just one file, as we saw in the previous exercise. You could imagine it would take way too long to process all of the data sequentially. Thankfully, we have [PyWren](https://www.youtube.com/watch?v=VSaDPc1Cs5U).

## 1. To the ~~Batmobile~~ Cloud!!!!!!

In [ ]:
# some libraries that are useful for this tutorial
import sys

# We need to load PyWren and create an executor instance
import pywren
pwex = pywren.default_executor()

## 2. Count
Let’s see how many records in total are in this data set.

In [ ]:
def count(filename):
    data = read_from_s3(wikipedia_bucket, filename)
    return (len(data.split("\n")) if data else 0)    

futures = pwex.map(count, filenames)
pywren.wait(futures)

result = sum([f.result() for f in futures])
print(result)

This should launch 73 PyWren tasks. After finishing the job, let's plot again to check the execution. This should look more interesting than the simple job before.

In [ ]:
plot_pywren_execution(futures)

# 3. Visits for English Pages
Recall from above when we peek the date, that the second field is the “project code” and contains information about the language of the pages. For example, the project code “en” indicates an English page. Let’s calculate the page counts of english pages, grouped by dates.

In [ ]:
from itertools import groupby
from operator import itemgetter
from functools import reduce

def aggregate_count(key_value_list):
    def reduce_f(obj1, obj2):
        return(obj1[0], obj1[1] + obj2[1])
    counts = [reduce(reduce_f, group) for _, group 
          in groupby(sorted(key_value_list), key=itemgetter(0))]
    
    return counts

def english_page_count(filename):
    data = read_from_s3(wikipedia_bucket, filename)
    # filter out the english pages
    en_pages = [d for d in data.split("\n") 
                if len(d.split(" ")) >= 4 and d.split(" ")[1] == "en"]
    # projection to create (date, pagecount) pairs
    en_kvpair_list = [(p.split(" ")[0][:8], int(p.split(" ")[3])) for p in en_pages]

    return aggregate_count(en_kvpair_list)
    
futures = pwex.map(english_page_count, filenames)
pywren.wait(futures)

results = [f.result() for f in futures]
en_page_counts_by_date = aggregate_count([x for y in results for x in y])
print(en_page_counts_by_date)